In [1]:
import pandas as pd
import clang.cindex
import tempfile
import os

class CodeToAST:
    def __init__(self):
        pass

    def full_ast(self, code):
        """
        Converts the given C code into its Abstract Syntax Tree (AST) representation.
        :param code: A string containing C code.
        :return: A string representation of the AST.
        """
        # Create a temporary file to store the C code
        with tempfile.NamedTemporaryFile(suffix=".c", delete=False) as tmp_file:
            tmp_file.write(code.encode())
            tmp_file.flush()
            tmp_file_path = tmp_file.name

        try:
            # Use libclang to parse the code and generate the AST
            index = clang.cindex.Index.create()
            translation_unit = index.parse(tmp_file_path)

            # Recursively build the AST string representation
            def build_ast_string(node, indent=0):
                ast_str = ""
                for child in node.get_children():
                    ast_str += '  ' * indent + f"{child.kind} {child.spelling}\n"
                    ast_str += build_ast_string(child, indent + 1)
                return ast_str

            return build_ast_string(translation_unit.cursor)

        finally:
            # Ensure the temporary file is removed after processing
            os.unlink(tmp_file_path)

    def ast_builder_csv(self, input_file, output_file):
        """
        Processes a CSV file with a 'code' column to generate ASTs for each code snippet.
        :param input_file: Path to the input CSV file containing a 'code' column.
        :param output_file: Path to save the output CSV file with the ASTs.
        """
        # Read the input CSV file
        data = pd.read_csv(input_file)

        # Ensure the CSV contains a 'code' column
        if 'code' not in data.columns:
            raise ValueError("Input CSV must contain a 'code' column.")

        # Convert each code snippet into its AST
        data['ast'] = data['code'].apply(self.full_ast)

        # Save the DataFrame with ASTs to the output file
        data.to_csv(output_file, index=False)
        print(f"ASTs have been successfully saved to {output_file}")

    def ast_for_single_code(self, code):
        """
        Generates the AST for a single C code snippet.
        :param code: A string containing C code.
        :return: A string representation of the AST.
        """
        return self.full_ast(code)



if __name__ == "__main__":
    # Instantiate the converter
    ast_converter = CodeToAST()

    # Example: Convert a CSV file
    input_csv_file = "input.csv"
    output_csv_file = "output.csv"
    ast_converter.ast_builder_csv(input_csv_file, output_csv_file)

    # Example: Convert a single code snippet
    single_code = "int main() { return 0; }"
    ast = ast_converter.ast_for_single_code(single_code)
    print("Generated AST:")
    print(ast)


FileNotFoundError: [Errno 2] No such file or directory: 'input.csv'